# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
Cities_DF = pd.read_csv('../City_Data.csv')

Cities_DF

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,12.67,84,91,5.75,NZ,1651313059
1,1,yazman,29.1212,71.7446,45.22,6,0,3.17,PK,1651313059
2,2,puerto ayora,-0.7393,-90.3518,21.40,93,49,2.68,EC,1651313060
3,3,mataura,-46.1927,168.8643,12.49,91,82,1.67,NZ,1651313060
4,4,cape town,-33.9258,18.4232,20.62,67,17,1.34,ZA,1651312820
...,...,...,...,...,...,...,...,...,...,...
551,551,denpasar,-8.6500,115.2167,27.81,88,40,4.63,ID,1651313186
552,552,amazar,53.8580,120.8750,7.28,29,99,1.76,RU,1651313314
553,553,umtata,-31.5889,28.7844,17.97,57,20,2.23,ZA,1651313314
554,554,grand-santi,4.2500,-54.3833,21.45,99,77,1.10,GF,1651313315


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key) 

# Store latitude and longitude in locations, and humidity as humidity
Locations = Cities_DF[['Lat', 'Lng']]
Humidity = Cities_DF['Humidity']

In [7]:
# Plot Heatmap
# center designates lat and lng center of map (had to do this because default settings were showing too much)
fig = gmaps.figure(center = (20,0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(Locations, weights=Humidity, 
                                 dissipating=False, max_intensity=max(Cities_DF['Humidity']),
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
Cloudiness = Cities_DF['Cloudiness']
Temperature = Cities_DF['Max Temp']
Wind_Speed = Cities_DF['Wind Speed']

Filtered_Cities_DF = Cities_DF.loc[(Cloudiness < 10)]
Filtered_Cities_DF = Filtered_Cities_DF.loc[(Temperature > 21) & (Temperature < 27)]
Filtered_Cities_DF = Filtered_Cities_DF.loc[(Wind_Speed < 4)]

Filtered_Cities_DF.dropna()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
82,82,hami,42.8000,93.4500,21.55,11,9,2.27,CN,1651313094
233,233,manakara,-22.1333,48.0167,26.80,60,3,2.19,MG,1651313168
317,317,mahanoro,-19.9000,48.8000,25.62,60,0,3.00,MG,1651313210
325,325,ponta do sol,32.6667,-17.1000,22.35,58,0,0.36,PT,1651313213
327,327,chitral,35.8462,71.7858,26.42,25,3,0.74,PK,1651313215
332,332,zhezkazgan,47.8043,67.7144,26.99,22,0,3.00,KZ,1651313217
425,425,vangaindrano,-23.3500,47.6000,26.10,65,2,0.73,MG,1651313257
516,516,barcelona,41.3888,2.1590,25.10,62,0,2.57,ES,1651313211


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:

Hotel_DF = Filtered_Cities_DF

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": 'lodging',
    "key": g_key}


for index, row in Hotel_DF.iterrows():
    # get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']

    # change location each iteration while leaving original params in place
    params['location'] = f"{lat},{lng}"
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        Hotel_DF.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print('Missing field/result... skipping.')
        
    print('------------')



Retrieving Results for Index 82: hami.
Closest hotel is Laoyutang Resort.
------------
Retrieving Results for Index 233: manakara.
Closest hotel is La Residence Manakara.
------------
Retrieving Results for Index 317: mahanoro.
Closest hotel is BUNGALOW MAHAMANINA.
------------
Retrieving Results for Index 325: ponta do sol.
Closest hotel is Hotel do Campo.
------------
Retrieving Results for Index 327: chitral.
Closest hotel is PTDC Motel Chitral.
------------
Retrieving Results for Index 332: zhezkazgan.
Closest hotel is Hotel Zumrat.
------------
Retrieving Results for Index 425: vangaindrano.
Closest hotel is Hotel Fanilo.
------------
Retrieving Results for Index 516: barcelona.
Closest hotel is Axel Hotel Barcelona.
------------


In [10]:
Hotel_DF

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
82,82,hami,42.8000,93.4500,21.55,11,9,2.27,CN,1651313094,Laoyutang Resort
233,233,manakara,-22.1333,48.0167,26.80,60,3,2.19,MG,1651313168,La Residence Manakara
317,317,mahanoro,-19.9000,48.8000,25.62,60,0,3.00,MG,1651313210,BUNGALOW MAHAMANINA
325,325,ponta do sol,32.6667,-17.1000,22.35,58,0,0.36,PT,1651313213,Hotel do Campo
327,327,chitral,35.8462,71.7858,26.42,25,3,0.74,PK,1651313215,PTDC Motel Chitral
332,332,zhezkazgan,47.8043,67.7144,26.99,22,0,3.00,KZ,1651313217,Hotel Zumrat
425,425,vangaindrano,-23.3500,47.6000,26.10,65,2,0.73,MG,1651313257,Hotel Fanilo
516,516,barcelona,41.3888,2.1590,25.10,62,0,2.57,ES,1651313211,Axel Hotel Barcelona


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in Hotel_DF.iterrows()]

locations = Hotel_DF[['Lat', 'Lng']]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))